### Stepwise Regression
Stepwise Regression is used for finding a regression algorithm for multiple independent variables that all affect a dependent variable in a different way. We can use a forward or backwards method in order to add/remove variables.

In [ ]:
# Imports
import torch
import torch.nn as nn

import numpy as np
import random
import time

import matplotlib.pyplot as plt

In [ ]:
# Setting torch to use GPU acceleration if possible.
device = torch.device("cpu")

if torch.cuda.is_available():
    device = torch.device("cuda")

torch.set_default_device(device)
print(f"Using device: {torch.get_default_device()}")

In [ ]:
# ====================== DATA COLLECTION ======================

In [ ]:
# Generate dummy data using numpy. The data here is guaranteed to have some correlation.
DATA_COUNT = 1000
X_SCALES = [0, 5, 2, -3, 0, 1, 6]
X_MULTIPLIER = 250

X = np.random.random((DATA_COUNT, len(X_SCALES))) * X_MULTIPLIER
Y = np.zeros(DATA_COUNT)

for i in range(DATA_COUNT):
    Y[i] = sum([X[i, j] * X_SCALES[j] for j in range(len(X_SCALES))])

In [ ]:
# Create the train and test splits.
TRAIN_SPLIT = 0.8

splitIndex = int(DATA_COUNT * TRAIN_SPLIT)

trainX = X[:splitIndex]
trainY = Y[:splitIndex]

testX = X[splitIndex:]
testY = Y[splitIndex:]

In [ ]:
# ====================== MODEL CONSTRUCTION - LINEAR MODEL ======================

In [ ]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        y = self.linear(x)
        return y

In [ ]:
# Train a new linear regression model, given the index of X.
def train_linear_model(X_index, log_loss = False):
    LEARNING_RATE = 1e-3
    EPOCHS = 1000
    BATCH_SIZE = 64
    REPORT_INTERVAL = 100
    
    agent = LinearRegressionModel()
    
    allLosses = []
    agent.train()
    
    lossFN = nn.MSELoss()
    optimizer = torch.optim.SGD(agent.parameters(), lr = LEARNING_RATE)
    
    for epoch in range(1, EPOCHS + 1):
        currentLoss = 0
        
        agent.zero_grad() # Reset Gradients.
    
        # Create batches.
        batches = list(range(len(trainX)))
        random.shuffle(batches)
        batches = np.array_split(batches, len(batches) // BATCH_SIZE)
    
        # Run through the batches.
        for i, batch in enumerate(batches):
            batchLoss = 0
    
            for index in batch:
                x = trainX[index, X_index]
                y = trainY[index]
    
                output = agent(torch.from_numpy(np.array([x])).float())
                loss = lossFN(output, torch.from_numpy(np.array([y])).float())
                batchLoss += loss
    
            # Batch complete. Optimise parameters.
            batchLoss.backward()
            nn.utils.clip_grad_norm_(agent.parameters(), 3)
            optimizer.step()
            optimizer.zero_grad()
            currentLoss += batchLoss.item() / len(batch)
    
        allLosses.append(currentLoss)
    
        if epoch % REPORT_INTERVAL == 0 and log_loss:
            print(f"Epoch #{epoch}: Average batch loss - {allLosses[-1]}")

    if log_loss:
        return agent, allLosses

    return agent

In [ ]:
results = train_linear_model(1, True)